In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam
from sklearn.metrics import confusion_matrix

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

#print("TensorFlow version: ", tf.__version__)
#assert version.parse(tf.__version__).release[0] >= 2, \
#    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


In [2]:
import gc

In [3]:
import random 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
# UPDATED: import from tensorflow.keras instead of keras
from tensorflow.keras import layers, optimizers, losses, metrics
import gc
import tensorflow_addons as tfa

In [4]:
oneClass = "Infiltration"

In [5]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(tf.cast(y_pred,tf.float32) - tf.cast(y_true,tf.float32)), axis=-1))

In [9]:
def construct_network1(dropout=0.08425517073874295, neuronPct=0.1767547775828121, neuronShrink=0.33180474398878285):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat1'))
    model.add(Dense(neuronCount,name='dense1'))
    model.add(Activation('relu',name='relu1'))
    model.add(Dropout(dropout, name='dropout1'))
    model.add(Dense(14, activation='sigmoid',name='midLayer1')) # Output
    weights_path=None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer1')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer1')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [10]:
def construct_network2(dropout=0.15672137551441198, neuronPct=0.2197894476507525, neuronShrink=0.3803316528497302, noisePct=0.282563134185142):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat2'))
    model.add(Dense(neuronCount,name='dense2'))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu',name='relu2'))
    model.add(Dropout(dropout, name='dropout2'))
    model.add(Dense(14, activation='sigmoid',name='midLayer2')) # Output
    weights_path=None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    #model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer2')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer2')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [11]:
def construct_network(model1,model2):
    model = Model([model1.input,model2.input], [model1.output,model2.output])
    return model 

In [12]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train"+oneClass, class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev"+oneClass, class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
keras.backend.clear_session()
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)
#print(str(train_steps))
#print(str(train_counts))
#print(str(batch_size))

** compute class weights from training data **
2281: 13914
431: 13914
2801: 13914
13914: 13914
822: 13914
1057: 13914
414: 13914
662: 13914
841: 13914
726: 13914
313: 13914
234: 13914
511: 13914
22: 13914
** class_weights **
[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}]


In [13]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 22433 **


In [14]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [15]:
def optimize_network(lr):
    gc.collect()
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    start_time = time.time()
    model1 = construct_network1()
    model2 = construct_network2()
    
    optimizer = SGD(lr=initial_learning_rate)
    
    alpha = 0.9340456763831478
    gamma = 1.4195808780694898
    model1.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss})

    alpha = 0.7297456293468533
    gamma = 1.2700405014991505
    model2.compile(optimizer=optimizer,loss={'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})
  
    model = construct_network(model1=model1,model2=model2)
    model.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss,'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})

    output_weights_path = os.path.join(output_dir,  str(lr)+"_"+output_weights_name)
    
    print(f"** set output weights path to: {output_weights_path} **")
                  
    
                  
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
  
    model.summary()
  
    callbacks = [
            checkpoint,
            #keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    ]
    
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=[class_weights,class_weights,class_weights,class_weights,class_weights,class_weights],
            workers=generator_workers,
            shuffle=False,
        )
        
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(lr)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    auprcs = []
    precision = dict()
    recall = dict()
    threshold = dict()
    with open(test_log_path, "w") as f:
        for k in range(6):
            for i in range(len(class_names)):
                 if(class_names[i] == str(oneClass)):
                
                    try:
                        score = roc_auc_score(y[:, i], y_hat[k][:, i])
                        precision[i], recall[i], threshold[i] = precision_recall_curve(y[:, i], y_hat[k][:, i])
                        tmp = auc(recall[i], precision[i])
                        aurocs.append(score)
                        auprcs.append(tmp) 
                    except ValueError:
                        score = 0
               
                    print(f"auroc {str(k)+class_names[i]}: {score}\n")
                    print(f"auprc {str(k)+class_names[i]}: {tmp}\n")
                    f.write(f"auroc {str(k)+class_names[i]}: {score}\n")
                    f.write(f"auprc {str(k)+class_names[i]}: {tmp}\n")
        
        mean_auroc = np.mean(aurocs)
        mean_auprc = float(np.mean(auprcs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}\n")
        f.write(f"mean auprc: {mean_auprc}\n")
        print(f"mean auprc: {mean_auprc}\n")
        
        max_auroc = np.max(aurocs)
        max_auprc = float(np.max(auprcs))
        f.write("-------------------------\n")
        f.write(f"max auroc: {max_auroc}\n")
        print(f"max auroc: {max_auroc}\n")
        f.write(f"max auprc: {max_auprc}\n")
        print(f"max auprc: {max_auprc}\n")
    
    keras.backend.clear_session()
    time_took = time.time() - start_time
    
    return max_auroc, time_took
    


In [16]:
step = np.arange(0.009, 0.0151, 0.001)
maxi = []
for k in np.nditer(step):
    opn, daTime = optimize_network(k)
    print(daTime)
    maxi.append(opn)




** set output weights path to: ./experiments/0.009_weights.h5 **
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
flat2 (Flatten)                 (None, 1536)         0           input_2[0][0]                    
__________________________________________________________________________________________________
flat1 (Flatten)                 (None, 1536)         0           input_1[0][0]                    
___________________________

Epoch 6/11
13901/13914 [============================>.] - ETA: 0s - loss: 4.0820 - leftLayer1_loss: 0.0436 - midLayer1_loss: 1.3640 - rightLayer1_loss: 0.6603 - leftLayer2_loss: 0.0308 - midLayer2_loss: 1.3397 - rightLayer2_loss: 0.6435
Epoch 00006: val_loss improved from 4.11820 to 4.09199, saving model to ./experiments/0.009_weights.h5
13914/13914 [==============================] - 35s 3ms/step - loss: 4.0824 - leftLayer1_loss: 0.0436 - midLayer1_loss: 1.3640 - rightLayer1_loss: 0.6604 - leftLayer2_loss: 0.0308 - midLayer2_loss: 1.3397 - rightLayer2_loss: 0.6437 - val_loss: 4.0920 - val_leftLayer1_loss: 0.0411 - val_midLayer1_loss: 1.3582 - val_rightLayer1_loss: 0.6414 - val_leftLayer2_loss: 0.0453 - val_midLayer2_loss: 1.3076 - val_rightLayer2_loss: 0.6985
Epoch 7/11
13902/13914 [============================>.] - ETA: 0s - loss: 4.0693 - leftLayer1_loss: 0.0402 - midLayer1_loss: 1.3642 - rightLayer1_loss: 0.6537 - leftLayer2_loss: 0.0297 - midLayer2_loss: 1.3406 - rightLayer2_loss: 

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 13914 steps, validate for 2018 steps
Epoch 1/11
13896/13914 [============================>.] - ETA: 0s - loss: 5.0611 - leftLayer1_loss: 0.1046 - midLayer1_loss: 1.3661 - rightLayer1_loss: 1.1845 - leftLayer2_loss: 0.0866 - midLayer2_loss: 1.3744 - rightLayer2_loss: 0.9450
Epoch 00001: val_loss improved from inf to 4.61443, saving model to ./experiments/0.009999999999999998_weights.h5
13914/13914 [==============================] - 36s 3ms/step - loss: 5.0607 - leftLayer1_loss: 0.1046 - midLayer1_loss: 1.3662 - rightLayer1_loss: 1.1842 - leftLayer2_loss: 0.0865 - midLayer2_loss: 1.3744 - rightLayer2_loss: 0.9448 - val_loss: 4.6144 - val_leftLayer1_loss: 0.0901 - val_midLayer1_loss: 1.3634 - val_rightLayer1_loss: 0.8533 - val_leftLayer2_loss: 0.0803 - val_midLayer2_loss: 1.3295 - val_rightLayer2_lo

13902/13914 [============================>.] - ETA: 0s - loss: 4.0741 - leftLayer1_loss: 0.0329 - midLayer1_loss: 1.3663 - rightLayer1_loss: 0.6412 - leftLayer2_loss: 0.0271 - midLayer2_loss: 1.3713 - rightLayer2_loss: 0.6353
Epoch 00011: val_loss improved from 4.06483 to 4.05638, saving model to ./experiments/0.009999999999999998_weights.h5
22433/22433 [==============================] - 27s 1ms/step
** write log to ./experiments/0.009999999999999998_test.log **
auroc 0Infiltration: 0.502374106023741

auprc 0Infiltration: 0.16746115101094922

auroc 1Infiltration: 0.4293642084919661

auprc 1Infiltration: 0.146463680759498

auroc 2Infiltration: 0.3829089945246206

auprc 2Infiltration: 0.13348486650964483

auroc 3Infiltration: 0.40482068575491076

auprc 3Infiltration: 0.1410009135448698

auroc 4Infiltration: 0.3810524195591275

auprc 4Infiltration: 0.13190004100307162

auroc 5Infiltration: 0.3630529835867682

auprc 5Infiltration: 0.1309564714179775

mean auroc: 0.41059556632352234

mean a

Epoch 4/11
13896/13914 [============================>.] - ETA: 0s - loss: 4.2774 - leftLayer1_loss: 0.0554 - midLayer1_loss: 1.2991 - rightLayer1_loss: 0.6888 - leftLayer2_loss: 0.0349 - midLayer2_loss: 1.5471 - rightLayer2_loss: 0.6521
Epoch 00004: val_loss improved from 4.23343 to 4.17619, saving model to ./experiments/0.010999999999999998_weights.h5
13914/13914 [==============================] - 34s 2ms/step - loss: 4.2779 - leftLayer1_loss: 0.0554 - midLayer1_loss: 1.2991 - rightLayer1_loss: 0.6890 - leftLayer2_loss: 0.0349 - midLayer2_loss: 1.5472 - rightLayer2_loss: 0.6523 - val_loss: 4.1762 - val_leftLayer1_loss: 0.0513 - val_midLayer1_loss: 1.2923 - val_rightLayer1_loss: 0.6658 - val_leftLayer2_loss: 0.0490 - val_midLayer2_loss: 1.3993 - val_rightLayer2_loss: 0.7185
Epoch 5/11
13902/13914 [============================>.] - ETA: 0s - loss: 4.2439 - leftLayer1_loss: 0.0485 - midLayer1_loss: 1.2991 - rightLayer1_loss: 0.6732 - leftLayer2_loss: 0.0320 - midLayer2_loss: 1.5441 - rig

...

13908/13914 [============================>.] - ETA: 0s - loss: 4.0006 - leftLayer1_loss: 0.0328 - midLayer1_loss: 1.3014 - rightLayer1_loss: 0.6388 - leftLayer2_loss: 0.0273 - midLayer2_loss: 1.3658 - rightLayer2_loss: 0.6346
Epoch 00011: val_loss improved from 3.94615 to 3.93818, saving model to ./experiments/0.011999999999999997_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.011999999999999997_test.log **
auroc 0Infiltration: 0.4118709145581877

auprc 0Infiltration: 0.14500530931553396

auroc 1Infiltration: 0.3992720981649742

auprc 1Infiltration: 0.13853827686065215

auroc 2Infiltration: 0.3935724121284616

auprc 2Infiltration: 0.1387344833488518

auroc 3Infiltration: 0.4313204356632975

auprc 3Infiltration: 0.14840658569212103

auroc 4Infiltration: 0.5982636378876643

auprc 4Infiltration: 0.2406326659418536

auroc 5Infiltration: 0.4107148913045418

auprc 5Infiltration: 0.14234193275818824

mean auroc: 0.44083573161785444

mean

Epoch 4/11
13910/13914 [============================>.] - ETA: 0s - loss: 4.2187 - leftLayer1_loss: 0.0540 - midLayer1_loss: 1.3859 - rightLayer1_loss: 0.6831 - leftLayer2_loss: 0.0362 - midLayer2_loss: 1.4041 - rightLayer2_loss: 0.6554
Epoch 00004: val_loss improved from 4.26100 to 4.20107, saving model to ./experiments/0.012999999999999996_weights.h5
13914/13914 [==============================] - 35s 2ms/step - loss: 4.2187 - leftLayer1_loss: 0.0540 - midLayer1_loss: 1.3859 - rightLayer1_loss: 0.6831 - leftLayer2_loss: 0.0362 - midLayer2_loss: 1.4041 - rightLayer2_loss: 0.6554 - val_loss: 4.2011 - val_leftLayer1_loss: 0.0499 - val_midLayer1_loss: 1.3749 - val_rightLayer1_loss: 0.6610 - val_leftLayer2_loss: 0.0529 - val_midLayer2_loss: 1.3154 - val_rightLayer2_loss: 0.7470
Epoch 5/11
13902/13914 [============================>.] - ETA: 0s - loss: 4.1887 - leftLayer1_loss: 0.0472 - midLayer1_loss: 1.3859 - rightLayer1_loss: 0.6685 - leftLayer2_loss: 0.0330 - midLayer2_loss: 1.4049 - rig

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 13914 steps, validate for 2018 steps
Epoch 1/11
13907/13914 [============================>.] - ETA: 0s - loss: 4.9902 - leftLayer1_loss: 0.1074 - midLayer1_loss: 1.3465 - rightLayer1_loss: 1.2202 - leftLayer2_loss: 0.0901 - midLayer2_loss: 1.3017 - rightLayer2_loss: 0.9242
Epoch 00001: val_loss improved from inf to 4.59230, saving model to ./experiments/0.013999999999999995_weights.h5
13914/13914 [==============================] - 34s 2ms/step - loss: 4.9901 - leftLayer1_loss: 0.1074 - midLayer1_loss: 1.3465 - rightLayer1_loss: 1.2201 - leftLayer2_loss: 0.0901 - midLayer2_loss: 1.3018 - rightLayer2_loss: 0.9242 - val_loss: 4.5923 - val_leftLayer1_loss: 0.0936 - val_midLayer1_loss: 1.3435 - val_rightLayer1_loss: 0.8804 - val_leftLayer2_loss: 0.0845 - val_midLayer2_loss: 1.2841 - val_rightLayer2_lo

13907/13914 [============================>.] - ETA: 0s - loss: 3.9848 - leftLayer1_loss: 0.0340 - midLayer1_loss: 1.3463 - rightLayer1_loss: 0.6433 - leftLayer2_loss: 0.0275 - midLayer2_loss: 1.2983 - rightLayer2_loss: 0.6355
Epoch 00011: val_loss improved from 4.00741 to 3.99833, saving model to ./experiments/0.013999999999999995_weights.h5
22433/22433 [==============================] - 27s 1ms/step
** write log to ./experiments/0.013999999999999995_test.log **
auroc 0Infiltration: 0.4312245592023759

auprc 0Infiltration: 0.1461606771739267

auroc 1Infiltration: 0.5885596507971422

auprc 1Infiltration: 0.214805773071767

auroc 2Infiltration: 0.4073538810744699

auprc 2Infiltration: 0.14006075509631263

auroc 3Infiltration: 0.40498719610573786

auprc 3Infiltration: 0.14029845306715868

auroc 4Infiltration: 0.4267308941471972

auprc 4Infiltration: 0.14312411996872287

auroc 5Infiltration: 0.44136885307011314

auprc 5Infiltration: 0.1493590106534098

mean auroc: 0.45003750573283935

mean

Epoch 4/11
13898/13914 [============================>.] - ETA: 0s - loss: 4.0950 - leftLayer1_loss: 0.0544 - midLayer1_loss: 1.3124 - rightLayer1_loss: 0.6835 - leftLayer2_loss: 0.0357 - midLayer2_loss: 1.3558 - rightLayer2_loss: 0.6533
Epoch 00004: val_loss improved from 4.18382 to 4.12611, saving model to ./experiments/0.014999999999999994_weights.h5
13914/13914 [==============================] - 34s 2ms/step - loss: 4.0954 - leftLayer1_loss: 0.0544 - midLayer1_loss: 1.3124 - rightLayer1_loss: 0.6837 - leftLayer2_loss: 0.0357 - midLayer2_loss: 1.3558 - rightLayer2_loss: 0.6535 - val_loss: 4.1261 - val_leftLayer1_loss: 0.0499 - val_midLayer1_loss: 1.3058 - val_rightLayer1_loss: 0.6592 - val_leftLayer2_loss: 0.0525 - val_midLayer2_loss: 1.3255 - val_rightLayer2_loss: 0.7333
Epoch 5/11
13894/13914 [============================>.] - ETA: 0s - loss: 4.0684 - leftLayer1_loss: 0.0476 - midLayer1_loss: 1.3126 - rightLayer1_loss: 0.6690 - leftLayer2_loss: 0.0327 - midLayer2_loss: 1.3590 - rig

In [17]:
print(np.max(maxi))

0.5982636378876643
